# News Retrieval
- same news retrieval from initial news (dailynews)

In [3]:
from pymilvus import MilvusClient

# client = MilvusClient("milvus/milvus_demo.db")
client = MilvusClient(uri="http://localhost:19530", token="root:Milvus")

In [4]:
client.has_collection("news_collection")

False

In [8]:
with open('milvus/data/dailynews.txt', 'r') as f:
    dailynews = f.read()
with open('milvus/data/thairath.txt', 'r') as f:
    thairath = f.read()
with open('milvus/data/pptv.txt', 'r') as f:
    pptv = f.read()
    
dailynews, thairath, pptv

('เมื่อวันที่ 22 ส.ค. ที่พารากอนฮอลล์ ห้างสรรพสินค้าสยามพารากอน สื่อในเครือเนชั่น จัดงาน Dinner Talk Vision for Thailand 2024 เปิดเวทีแสดงวิสัยทัศน์ครั้งสำคัญ ของนายทักษิณ ชินวัตร อดีตนายกรัฐมนตรี เพื่อรู้ถึงทิศทางเศรษฐกิจจะขับเคลื่อนไปอย่างไร และทิศทางการเมืองไทยจากนี้ต่อไป ทันทีที่นายทักษิณ เดินทางมาถึง ได้เข้ามานั่งพักที่ SCBX พร้อมพูดคุยกับนักธุรกิจ และนักการเมืองที่เดินทางมาร่วมงาน อาทิ นายภูมิธรรม เวชยชัย รองนายกรัฐมนตรี และรมว.พาณิชย์ นายสุริยะ จึงรุ่งเรืองกิจ รองนายกรัฐมนตรี และรมว.คมนาคม นายจุลพันธ์ อมรวิวัฒน์ รมช.คลัง นพ.พรหมินทร์ เลิศสุริย์เดช อดีตเลขาธิการนายกรัฐมนตรี นายสุวัจน์ ลิปตพัลลภ ประธานพรรคชาติพัฒนา นายธนินท์ เจียรวนนท์ อดีตประธานกรรมการของเครือเจริญโภคภัณฑ์ และนายคีรี กาญจนพาสน์ ประธานกรรมการบริษัท บีทีเอส กรุ๊ป โฮลดิ้งส์ จำกัด หลังจากนั้น นายทักษิณ ได้เดินขึ้นมาที่ด้านหน้าบริเวณงาน และรับชมนิทรรศการที่จัดขึ้น พร้อมถ่ายภาพร่วมกันกับนักการเมือง นักธุรกิจ และคณะผู้บริหารในเครือเนชั่น ผู้สื่อข่าวรายงานเพิ่มเติมว่า นักการเมืองมาร่วมงานคึกคัก อาทิ นายวราวุธ ศิลปอาชา รม

In [9]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = t

In [20]:
import json
import os

with open('milvus/data/news.json', 'r') as f:
    news = json.load(f)
    json_datas = [d['data']for d in news[:3]]

In [21]:
json_datas

['วันที่ 13 ก.ย. น.ส.กาญจนา โชคไพศาลศิลป์ ผู้บริหารงานวิจัย ศูนย์วิจัยกสิกรไทย เปิดเผยว่า เงินบาทแตะระดับแข็งค่าสุดในรอบ 19 เดือนครั้งใหม่ ที่ 33.32 บาทต่อดอลลาร์ (แข็งค่าสุดนับตั้งแต่ต้นเดือนกุมภาพันธ์ 2566) ก่อนจะกลับมาปรับตัวอยู่ที่ระดับ 33.33-33.35 บาทต่อดอลลาร์ ในช่วงเช้าวันนี้ (08.35 น.) แข็งค่าขึ้นเมื่อเทียบกับระดับปิดตลาดวานนี้ที่ 33.75 บาทต่อดอลลาร์ ทั้งนี้ เงินบาทขยับแข็งค่าขึ้น โดยมีแรงหนุนสำคัญจากการพุ่งขึ้นของราคาทองคำในตลาดโลกที่ทำสถิติสูงสุดเป็นประวัติการณ์ครั้งใหม่ ประกอบกับน่าจะมีแรงหนุนเพิ่มเติมจากแรงขายเงินดอลลาร์ โดยเฉพาะเมื่อเทียบกับเงินยูโร หลังการประชุมธนาคารกลางยุโรป เพราะแม้ธนาคารกลางยุโรป จะมีการปรับลดอัตราดอกเบี้ยในการประชุมรอบนี้ลง 0.25% ตามที่ตลาดคาด แต่ยังไม่มีสัญญาณลดดอกเบี้ยในการประชุมรอบหน้าในเดือนตุลาคม นอกจากนี้ เงินดอลลาร์ ยังมีปัจจัยลบจากตัวเลขเศรษฐกิจสหรัฐ ทั้งจำนวนผู้ขอรับสวัสดิการว่างงานรายสัปดาห์ และดัชนีราคาผู้ผลิตที่ออกมาต่ำกว่าตัวเลขคาดการณ์ของตลาดด้วยเช่นกัน สำหรับกรอบการเคลื่อนไหวของเงินบาทในวันนี้ ประเมินเบื้องต้นไว้ที่ 33.25-33.55 บาทต่อด

In [22]:
docs = json_datas + [
    dailynews,
    thairath,
    pptv,
]

print(len(docs))

docs_embeddings = bge_m3_ef.encode_documents(docs)

6


In [23]:
print("Embeddings:", docs_embeddings)
print("Embeddings:", docs_embeddings.keys())
print("Embeddings:", len(docs_embeddings['dense']))
print("Embeddings:", docs_embeddings['sparse'].shape)
print("Dense document dim:", bge_m3_ef.dim["dense"], docs_embeddings["dense"][0].shape)
print("Sparse document dim:", bge_m3_ef.dim["sparse"], list(docs_embeddings["sparse"])[0].shape)

Embeddings: {'dense': [array([ 7.6412565e-05, -1.3311121e-02, -5.6907497e-02, ...,
        5.0507262e-03,  2.3743175e-02,  2.8348135e-02], dtype=float32), array([ 0.04638588,  0.00795219,  0.00039457, ...,  0.02083882,
       -0.01528262, -0.03445143], dtype=float32), array([ 5.8678526e-04,  1.7267155e-02, -5.2910894e-02, ...,
       -4.2016240e-05,  5.6884892e-04,  1.7930957e-02], dtype=float32), array([-0.06225254, -0.02033746, -0.00979749, ..., -0.0095912 ,
       -0.02146981, -0.04978221], dtype=float32), array([-0.04055409,  0.00907774, -0.00419278, ..., -0.01465963,
       -0.05719653,  0.01255647], dtype=float32), array([-0.04230585,  0.00783882, -0.03541169, ..., -0.0194636 ,
       -0.01921331, -0.06389072], dtype=float32)], 'sparse': <6x250002 sparse matrix of type '<class 'numpy.float64'>'
	with 1405 stored elements in Compressed Sparse Row format>}
Embeddings: dict_keys(['dense', 'sparse'])
Embeddings: 6
Embeddings: (6, 250002)
Dense document dim: 1024 (1024,)
Sparse docume

In [24]:
json_data_keys = (['data', 'category', 'date', 'publisher', 'url'])

In [25]:
data = [
    {"id": i, "vector": docs_embeddings['dense'][i], "text": docs[i], "category": "politics"}
    for i in range(len(docs))
]

In [26]:
data

[{'id': 0,
  'vector': array([ 7.6412565e-05, -1.3311121e-02, -5.6907497e-02, ...,
          5.0507262e-03,  2.3743175e-02,  2.8348135e-02], dtype=float32),
  'text': 'วันที่ 13 ก.ย. น.ส.กาญจนา โชคไพศาลศิลป์ ผู้บริหารงานวิจัย ศูนย์วิจัยกสิกรไทย เปิดเผยว่า เงินบาทแตะระดับแข็งค่าสุดในรอบ 19 เดือนครั้งใหม่ ที่ 33.32 บาทต่อดอลลาร์ (แข็งค่าสุดนับตั้งแต่ต้นเดือนกุมภาพันธ์ 2566) ก่อนจะกลับมาปรับตัวอยู่ที่ระดับ 33.33-33.35 บาทต่อดอลลาร์ ในช่วงเช้าวันนี้ (08.35 น.) แข็งค่าขึ้นเมื่อเทียบกับระดับปิดตลาดวานนี้ที่ 33.75 บาทต่อดอลลาร์ ทั้งนี้ เงินบาทขยับแข็งค่าขึ้น โดยมีแรงหนุนสำคัญจากการพุ่งขึ้นของราคาทองคำในตลาดโลกที่ทำสถิติสูงสุดเป็นประวัติการณ์ครั้งใหม่ ประกอบกับน่าจะมีแรงหนุนเพิ่มเติมจากแรงขายเงินดอลลาร์ โดยเฉพาะเมื่อเทียบกับเงินยูโร หลังการประชุมธนาคารกลางยุโรป เพราะแม้ธนาคารกลางยุโรป จะมีการปรับลดอัตราดอกเบี้ยในการประชุมรอบนี้ลง 0.25% ตามที่ตลาดคาด แต่ยังไม่มีสัญญาณลดดอกเบี้ยในการประชุมรอบหน้าในเดือนตุลาคม นอกจากนี้ เงินดอลลาร์ ยังมีปัจจัยลบจากตัวเลขเศรษฐกิจสหรัฐ ทั้งจำนวนผู้ขอรับสวัสดิการว่า

In [44]:
collection_name = "bge_collection"

if client.has_collection(collection_name="bge_collection"):
    client.drop_collection(collection_name="bge_collection")
    
client.create_collection(
    collection_name="bge_collection",
    dimension=1024,
)

In [45]:
res = client.insert(collection_name="bge_collection", data=data)

print(res)

{'insert_count': 6, 'ids': [0, 1, 2, 3, 4, 5]}


In [33]:
question = dailynews

search_res = client.search(
    collection_name=collection_name,
    data=[
        bge_m3_ef.encode_documents([question])["dense"][0]
    ],  
    limit=10,  # Return top 3 results
    # search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)


In [34]:
print(search_res)

data: ['[{\'id\': 3, \'distance\': 1.000000238418579, \'entity\': {\'text\': \'เมื่อวันที่ 22 ส.ค. ที่พารากอนฮอลล์ ห้างสรรพสินค้าสยามพารากอน สื่อในเครือเนชั่น จัดงาน Dinner Talk Vision for Thailand 2024 เปิดเวทีแสดงวิสัยทัศน์ครั้งสำคัญ ของนายทักษิณ ชินวัตร อดีตนายกรัฐมนตรี เพื่อรู้ถึงทิศทางเศรษฐกิจจะขับเคลื่อนไปอย่างไร และทิศทางการเมืองไทยจากนี้ต่อไป ทันทีที่นายทักษิณ เดินทางมาถึง ได้เข้ามานั่งพักที่ SCBX พร้อมพูดคุยกับนักธุรกิจ และนักการเมืองที่เดินทางมาร่วมงาน อาทิ นายภูมิธรรม เวชยชัย รองนายกรัฐมนตรี และรมว.พาณิชย์ นายสุริยะ จึงรุ่งเรืองกิจ รองนายกรัฐมนตรี และรมว.คมนาคม นายจุลพันธ์ อมรวิวัฒน์ รมช.คลัง นพ.พรหมินทร์ เลิศสุริย์เดช อดีตเลขาธิการนายกรัฐมนตรี นายสุวัจน์ ลิปตพัลลภ ประธานพรรคชาติพัฒนา นายธนินท์ เจียรวนนท์ อดีตประธานกรรมการของเครือเจริญโภคภัณฑ์ และนายคีรี กาญจนพาสน์ ประธานกรรมการบริษัท บีทีเอส กรุ๊ป โฮลดิ้งส์ จำกัด หลังจากนั้น นายทักษิณ ได้เดินขึ้นมาที่ด้านหน้าบริเวณงาน และรับชมนิทรรศการที่จัดขึ้น พร้อมถ่ายภาพร่วมกันกับนักการเมือง นักธุรกิจ และคณะผู้บริหารในเครือเนชั่น ผู้สื่

In [39]:
search_res[0]

[{'id': 3,
  'distance': 1.000000238418579,
  'entity': {'text': 'เมื่อวันที่ 22 ส.ค. ที่พารากอนฮอลล์ ห้างสรรพสินค้าสยามพารากอน สื่อในเครือเนชั่น จัดงาน Dinner Talk Vision for Thailand 2024 เปิดเวทีแสดงวิสัยทัศน์ครั้งสำคัญ ของนายทักษิณ ชินวัตร อดีตนายกรัฐมนตรี เพื่อรู้ถึงทิศทางเศรษฐกิจจะขับเคลื่อนไปอย่างไร และทิศทางการเมืองไทยจากนี้ต่อไป ทันทีที่นายทักษิณ เดินทางมาถึง ได้เข้ามานั่งพักที่ SCBX พร้อมพูดคุยกับนักธุรกิจ และนักการเมืองที่เดินทางมาร่วมงาน อาทิ นายภูมิธรรม เวชยชัย รองนายกรัฐมนตรี และรมว.พาณิชย์ นายสุริยะ จึงรุ่งเรืองกิจ รองนายกรัฐมนตรี และรมว.คมนาคม นายจุลพันธ์ อมรวิวัฒน์ รมช.คลัง นพ.พรหมินทร์ เลิศสุริย์เดช อดีตเลขาธิการนายกรัฐมนตรี นายสุวัจน์ ลิปตพัลลภ ประธานพรรคชาติพัฒนา นายธนินท์ เจียรวนนท์ อดีตประธานกรรมการของเครือเจริญโภคภัณฑ์ และนายคีรี กาญจนพาสน์ ประธานกรรมการบริษัท บีทีเอส กรุ๊ป โฮลดิ้งส์ จำกัด หลังจากนั้น นายทักษิณ ได้เดินขึ้นมาที่ด้านหน้าบริเวณงาน และรับชมนิทรรศการที่จัดขึ้น พร้อมถ่ายภาพร่วมกันกับนักการเมือง นักธุรกิจ และคณะผู้บริหารในเครือเนชั่น ผู้สื่อข่าวรายงานเพ

In [40]:
import json

retrieved_lines_with_distances = [
    (res["id"], res['entity']['text'], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        3,
        "เมื่อวันที่ 22 ส.ค. ที่พารากอนฮอลล์ ห้างสรรพสินค้าสยามพารากอน สื่อในเครือเนชั่น จัดงาน Dinner Talk Vision for Thailand 2024 เปิดเวทีแสดงวิสัยทัศน์ครั้งสำคัญ ของนายทักษิณ ชินวัตร อดีตนายกรัฐมนตรี เพื่อรู้ถึงทิศทางเศรษฐกิจจะขับเคลื่อนไปอย่างไร และทิศทางการเมืองไทยจากนี้ต่อไป ทันทีที่นายทักษิณ เดินทางมาถึง ได้เข้ามานั่งพักที่ SCBX พร้อมพูดคุยกับนักธุรกิจ และนักการเมืองที่เดินทางมาร่วมงาน อาทิ นายภูมิธรรม เวชยชัย รองนายกรัฐมนตรี และรมว.พาณิชย์ นายสุริยะ จึงรุ่งเรืองกิจ รองนายกรัฐมนตรี และรมว.คมนาคม นายจุลพันธ์ อมรวิวัฒน์ รมช.คลัง นพ.พรหมินทร์ เลิศสุริย์เดช อดีตเลขาธิการนายกรัฐมนตรี นายสุวัจน์ ลิปตพัลลภ ประธานพรรคชาติพัฒนา นายธนินท์ เจียรวนนท์ อดีตประธานกรรมการของเครือเจริญโภคภัณฑ์ และนายคีรี กาญจนพาสน์ ประธานกรรมการบริษัท บีทีเอส กรุ๊ป โฮลดิ้งส์ จำกัด หลังจากนั้น นายทักษิณ ได้เดินขึ้นมาที่ด้านหน้าบริเวณงาน และรับชมนิทรรศการที่จัดขึ้น พร้อมถ่ายภาพร่วมกันกับนักการเมือง นักธุรกิจ และคณะผู้บริหารในเครือเนชั่น ผู้สื่อข่าวรายงานเพิ่มเติมว่า นักการเมืองมาร่วมงานคึกคัก 